micrometrics    

In [11]:
import pandas as pd 
import numpy as np
import statsmodels.api as sm
from scipy.stats import norm
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


import data and setup 
#1

In [12]:
data = pd.read_csv('/Users/jadenfix/Desktop/Graduate School Materials/micrometrics/fl89-91eco526.csv')
data=pd.DataFrame(data)
data.head()

,alcohol,dmage,dmar,dmeduc,foreignb,pldel,tobacco,mblack,motherr,mhispan,dead,adequacy
0,1,30,0,15,0,1,0,0,0,0,0,2
1,0,14,1,9,0,1,0,1,0,0,0,0
2,0,30,1,13,0,1,0,1,0,0,0,2
3,0,22,0,12,0,1,0,0,0,0,0,2
4,0,27,0,16,0,1,0,1,0,0,0,2


In [13]:

# Add a squared term for age
data['dmagesqr'] = data['dmage'] ** 2

# Ensure categorical variables are treated correctly
data['adequacy'] = data['adequacy'].astype('category')
data['const'] = 1  # Add a constant term

# Define independent variables (X) including all specified covariates
X = data[['const', 'dmage', 'dmagesqr', 'dmeduc', 'dmar', 'mblack', 'mhispan', 'motherr', 'foreignb', 'tobacco', 'alcohol']]

# Define the dependent variable
y = data['dead']

# Fit the probit model
probit_model = sm.Probit(y, X)
probit_results = probit_model.fit()

# Display the summary
print(probit_results.summary())

# Perform Likelihood Ratio Test
llr = probit_results.llr  # Likelihood Ratio Test statistic
llr_pval = probit_results.llr_pvalue  # p-value for the test

# Print test results
print(f"Likelihood Ratio Test statistic: {llr:.2f}")
print(f"p-value: {llr_pval:.4f}")

Optimization terminated successfully.
         Current function value: 0.047795
         Iterations 8
                          Probit Regression Results                           
Dep. Variable:                   dead   No. Observations:               565943
Model:                         Probit   Df Residuals:                   565932
Method:                           MLE   Df Model:                           10
Date:                Wed, 22 Jan 2025   Pseudo R-squ.:                 0.02093
Time:                        16:20:47   Log-Likelihood:                -27049.
converged:                       True   LL-Null:                       -27627.
Covariance Type:            nonrobust   LLR p-value:                3.193e-242
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.9197      0.093    -20.685      0.000      -2.102      -1.738
dmage         -0.0298      0.

We can infer that everything is signficant at the 5% level except for mother hispanic.

In [21]:
# Calculate percentage of correctly predicted observations
predicted_probs = probit_results.predict(X)  # Predicted probabilities
predicted_classes = (predicted_probs > 0.5).astype(int)  # Convert probabilities to classes
accuracy = (predicted_classes == y).mean()  # Compare predictions with actual outcomes
print(f"Percentage of observations correctly predicted: {accuracy * 100:.2f}%")

# (c) Average effects (marginal effects)
# Calculate average marginal effects across all observations
marginal_effects = probit_results.get_margeff(at='overall').summary()
print("\nAverage Marginal Effects:")
print(marginal_effects)

# (d) Marginal effects at the mean of the covariates
# Calculate marginal effects at the mean values of the covariates
marginal_effects_mean = probit_results.get_margeff(at='mean').summary()
print("\nMarginal Effects at the Mean:")
print(marginal_effects_mean)

# (e) First differences for tobacco and alcohol
# Start with mean values for all covariates
mean_values = X.drop(columns=['const']).mean()  # Exclude constant from mean calculation
mean_values['const'] = 1  # Add constant term explicitly

# Convert mean values to a DataFrame
mean_df = pd.DataFrame([mean_values], columns=X.columns)

# First difference for tobacco
mean_df['tobacco'] = 1  # Set tobacco use
prob_with_tobacco = probit_results.predict(mean_df)

mean_df['tobacco'] = 0  # Set no tobacco use
prob_without_tobacco = probit_results.predict(mean_df)

first_diff_tobacco = prob_with_tobacco.iloc[0] - prob_without_tobacco.iloc[0]
print(f"First difference for tobacco: {first_diff_tobacco:.4f}")

# First difference for alcohol
mean_df['alcohol'] = 1  # Set alcohol use
prob_with_alcohol = probit_results.predict(mean_df)

mean_df['alcohol'] = 0  # Set no alcohol use
prob_without_alcohol = probit_results.predict(mean_df)

first_diff_alcohol = prob_with_alcohol.iloc[0] - prob_without_alcohol.iloc[0]
print(f"First difference for alcohol: {first_diff_alcohol:.4f}")

Percentage of observations correctly predicted: 99.15%

Average Marginal Effects:
       Probit Marginal Effects       
Dep. Variable:                   dead
Method:                          dydx
At:                           overall
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
dmage         -0.0007      0.000     -4.161      0.000      -0.001      -0.000
dmagesqr    1.196e-05   2.96e-06      4.037      0.000    6.15e-06    1.78e-05
dmeduc        -0.0004   5.62e-05     -7.057      0.000      -0.001      -0.000
dmar           0.0022      0.000      7.306      0.000       0.002       0.003
mblack         0.0058      0.000     18.419      0.000       0.005       0.006
mhispan        0.0006      0.000      1.124      0.261      -0.000       0.002
motherr        0.0033      0.001      3.132      0.002       0.001       0.005
foreignb      -0.0014      0.000     -3.185      0.001 

problem 4: logit

In [22]:
# Fit a logit model
logit_model = sm.Logit(y, X)
logit_results = logit_model.fit()

# Display the summary
print(logit_results.summary())


Optimization terminated successfully.
         Current function value: 0.047800
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                   dead   No. Observations:               565943
Model:                          Logit   Df Residuals:                   565932
Method:                           MLE   Df Model:                           10
Date:                Wed, 22 Jan 2025   Pseudo R-squ.:                 0.02083
Time:                        16:29:23   Log-Likelihood:                -27052.
converged:                       True   LL-Null:                       -27627.
Covariance Type:            nonrobust   LLR p-value:                5.076e-241
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.5471      0.246    -14.416      0.000      -4.029      -3.065
dmage         -0.0769      0.

a-b

In [23]:
# Average Marginal Effects (AME)
logit_marginal_effects = logit_results.get_margeff(at='overall').summary()
print("\nLogit Model Average Marginal Effects:")
print(logit_marginal_effects)

# Compare coefficients and scaling between Probit and Logit
coeff_comparison = pd.DataFrame({
    'Probit': probit_results.params,
    'Logit': logit_results.params,
    'Ratio (Logit/Probit)': logit_results.params / probit_results.params
})
print("\nProbit vs. Logit Coefficients:")
print(coeff_comparison)
print(pd.DataFrame({
    'Probit': probit_results.params,
    'Logit': logit_results.params
}))


Logit Model Average Marginal Effects:
        Logit Marginal Effects       
Dep. Variable:                   dead
Method:                          dydx
At:                           overall
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
dmage         -0.0006      0.000     -4.019      0.000      -0.001      -0.000
dmagesqr    1.135e-05   2.93e-06      3.868      0.000     5.6e-06    1.71e-05
dmeduc        -0.0004    5.6e-05     -7.118      0.000      -0.001      -0.000
dmar           0.0022      0.000      7.209      0.000       0.002       0.003
mblack         0.0057      0.000     18.249      0.000       0.005       0.006
mhispan        0.0005      0.000      1.093      0.275      -0.000       0.002
motherr        0.0034      0.001      3.181      0.001       0.001       0.006
foreignb      -0.0014      0.000     -3.242      0.001      -0.002      -0.001
tobacco        0.00

C

In [24]:
# Find the age that minimizes the probability of infant mortality
age_coeff = logit_results.params['dmage']
age_sq_coeff = logit_results.params['dmagesqr']
min_age = -age_coeff / (2 * age_sq_coeff)

# Variance and confidence interval
age_var = logit_results.cov_params().loc[['dmage', 'dmagesqr'], ['dmage', 'dmagesqr']]
gradient = np.array([1, 2 * min_age])
var_min_age = gradient @ age_var @ gradient.T
ci_min_age = [min_age - 1.96 * np.sqrt(var_min_age), min_age + 1.96 * np.sqrt(var_min_age)]

print(f"Minimum infant mortality occurs at age: {min_age:.2f}")
print(f"95% CI for minimum age: [{ci_min_age[0]:.2f}, {ci_min_age[1]:.2f}]")

Minimum infant mortality occurs at age: 28.38
95% CI for minimum age: [28.37, 28.38]


D  

In [28]:
# Define specific mother characteristics for prediction
specific_values = {
    'const': 1,
    'dmage': 25,
    'dmagesqr': 25**2,
    'dmeduc': 12,
    'dmar': 1,       # Unmarried
    'mblack': 0,     # Not black
    'mhispan': 0,    # Not Hispanic
    'motherr': 0,    # Not other race
    'foreignb': 0,   # Not foreign-born
    'tobacco': 1,    # Smoked during pregnancy
    'alcohol': 0     # Did not drink during pregnancy
}

# Convert to DataFrame to match input requirements
specific_df = pd.DataFrame([specific_values], columns=X.columns)

# Predict probability for specific characteristics
specific_prob = logit_results.predict(specific_df)

# Confidence interval for the prediction
cov_matrix = logit_results.cov_params()
gradient = specific_df.values.flatten()
predicted_var = gradient @ cov_matrix @ gradient.T
ci_prob = [
    specific_prob[0] - 1.96 * np.sqrt(predicted_var),
    specific_prob[0] + 1.96 * np.sqrt(predicted_var)
]

print(f"Predicted probability of infant mortality: {specific_prob[0]:.4f}")
print(f"95% Confidence Interval: [{ci_prob[0]:.4f}, {ci_prob[1]:.4f}]")

Predicted probability of infant mortality: 0.0108
95% Confidence Interval: [-0.0687, 0.0902]


E

In [30]:
# Marginal effects for a mother with covariates at the mean
marginal_effects_mean = logit_results.get_margeff(at='mean').summary()
print("\nMarginal Effects at the Mean (Logit):")
print(marginal_effects_mean)

# Marginal effects for a specific mother profile
specific_case = {
    'const': 1,
    'dmage': 28,
    'dmagesqr': 28**2,
    'dmeduc': 17,
    'dmar': 0,        # Married
    'mblack': 0,      # Not black
    'mhispan': 0,     # Not Hispanic
    'motherr': 0,     # Not other race
    'foreignb': 0,    # Not foreign-born
    'tobacco': 0,     # Did not smoke
    'alcohol': 0      # Did not drink
}

# Convert specific values to DataFrame
specific_case_df = pd.DataFrame([specific_case], columns=X.columns)

# Use atexog to calculate marginal effects for the specific case
specific_marginal_effects = logit_results.get_margeff(atexog=specific_case_df).summary()
print("\nMarginal Effects for Specific Case:")
print(specific_marginal_effects)


Marginal Effects at the Mean (Logit):
        Logit Marginal Effects       
Dep. Variable:                   dead
Method:                          dydx
At:                              mean
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
dmage         -0.0006      0.000     -4.028      0.000      -0.001      -0.000
dmagesqr    1.019e-05   2.63e-06      3.876      0.000    5.04e-06    1.53e-05
dmeduc        -0.0004   4.98e-05     -7.191      0.000      -0.000      -0.000
dmar           0.0020      0.000      7.272      0.000       0.001       0.003
mblack         0.0051      0.000     19.259      0.000       0.005       0.006
mhispan        0.0005      0.000      1.093      0.275      -0.000       0.001
motherr        0.0031      0.001      3.187      0.001       0.001       0.005
foreignb      -0.0013      0.000     -3.248      0.001      -0.002      -0.000
tobacco        0.00

F

In [27]:
# Fit a linear probability model with robust standard errors
lpm = sm.OLS(y, X).fit(cov_type='HC3')

# Display LPM summary
print("\nLinear Probability Model (LPM) Summary:")
print(lpm.summary())

# Extract average effects for comparison
lpm_avg_effects = lpm.params.drop('const')  # Coefficients from LPM
probit_avg_effects = probit_results.get_margeff(at='overall').margeff  # Marginal effects for Probit
logit_avg_effects = logit_results.get_margeff(at='overall').margeff  # Marginal effects for Logit

# Create a DataFrame for comparison
effects_comparison = pd.DataFrame({
    'LPM': lpm_avg_effects,
    'Probit': probit_avg_effects,
    'Logit': logit_avg_effects
})
print("\nAverage Effects Comparison (LPM vs. Probit vs. Logit):")
print(effects_comparison)

# Prediction for the specific case using LPM
specific_case_prob_lpm = lpm.predict(specific_case_df)

# Calculate confidence intervals for LPM prediction
specific_case_prob_lpm_var = specific_case_df.values.flatten() @ lpm.cov_params() @ specific_case_df.values.flatten().T
ci_prob_lpm = [
    specific_case_prob_lpm.iloc[0] - 1.96 * np.sqrt(specific_case_prob_lpm_var),
    specific_case_prob_lpm.iloc[0] + 1.96 * np.sqrt(specific_case_prob_lpm_var)
]

print(f"LPM Predicted probability: {specific_case_prob_lpm.iloc[0]:.4f}")
print(f"LPM 95% Confidence Interval: [{ci_prob_lpm[0]:.4f}, {ci_prob_lpm[1]:.4f}]")


Linear Probability Model (LPM) Summary:
                            OLS Regression Results                            
Dep. Variable:                   dead   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     89.98
Date:                Wed, 22 Jan 2025   Prob (F-statistic):          1.01e-186
Time:                        16:30:27   Log-Likelihood:             5.5033e+05
No. Observations:              565943   AIC:                        -1.101e+06
Df Residuals:                  565932   BIC:                        -1.101e+06
Df Model:                          10                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const      